<a href="https://colab.research.google.com/github/velumahalingam/Velu-Demo/blob/master/Velu_H2O_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print ("Hello Velu")

Hello Velu


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
## Mount google drive 

!mkdir -p drive 
!google-drive-ocamlfuse drive

In [0]:
import os


In [5]:
os.getcwd()

'/content'

In [0]:
os.chdir('/content/drive/ColabNotebooks/Code_Test')

In [9]:
os.getcwd()

'/content/drive/ColabNotebooks/Code_Test'

In [10]:
ls

C4V.csv            higgs_train_10k.csv  TimeSeriesAnalysisWithPython/
higgs_test_5k.csv  PerseusCaptoNew.csv  veludemo.ipynb


In [12]:
!pip install h2o

     |████████████████████████████████| 123.4MB 95kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.1-py2.py3-none-any.whl size=123410554 sha256=c8537ec086ef2a85a96dc573a29a9e69b022d675f18d3062382a8eb1f60d71cf
  Stored in directory: /root/.cache/pip/wheels/ff/82/f4/3fe9c895b93c53b25ae44dc44aa54dad571c711f48bc485fe9
Successfully built h2o


In [13]:
import h2o
from h2o.automl import H2OAutoML, get_leaderboard

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.5" 2019-10-15; OpenJDK Runtime Environment (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04); OpenJDK 64-Bit Server VM (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplmjiy_1c
  JVM stdout: /tmp/tmplmjiy_1c/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmplmjiy_1c/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_unknownUser_5ochwq
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [15]:
import os
os.getcwd()

'/content/drive/ColabNotebooks/Code_Test'

In [0]:
# Import a sample binary outcome train/test set into H2O
import pandas as pd
import csv


In [20]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("/content/drive/ColabNotebooks/Code_Test/higgs_train_10k.csv")
test = h2o.import_file("/content/drive/ColabNotebooks/Code_Test/higgs_test_5k.csv")



Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

In [0]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [23]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [24]:
# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20191226_133845,0.789613,0.551457,0.805056,0.308972,0.432207,0.186803,4920,0.335826
StackedEnsemble_BestOfFamily_AutoML_20191226_133845,0.788912,0.552287,0.803328,0.307921,0.432483,0.187041,2128,0.101815
XGBoost_grid__1_AutoML_20191226_133845_model_3,0.784783,0.558539,0.802597,0.328578,0.435036,0.189256,4019,0.008126
XGBoost_grid__1_AutoML_20191226_133845_model_1,0.784653,0.5581,0.802569,0.319823,0.434987,0.189214,6092,0.012309
XGBoost_3_AutoML_20191226_133845,0.783759,0.558382,0.80096,0.322163,0.435252,0.189445,4351,0.011641
XGBoost_grid__1_AutoML_20191226_133845_model_4,0.783664,0.557696,0.802604,0.33427,0.43517,0.189373,4297,0.012192
GBM_5_AutoML_20191226_133845,0.78219,0.558353,0.800234,0.319658,0.435512,0.18967,2306,0.016959
XGBoost_2_AutoML_20191226_133845,0.782155,0.557366,0.801728,0.337128,0.435331,0.189514,7132,0.012597
XGBoost_1_AutoML_20191226_133845,0.780395,0.5593,0.800388,0.322149,0.436235,0.190301,6991,0.012304
GBM_2_AutoML_20191226_133845,0.777673,0.562514,0.796181,0.334056,0.437583,0.191479,1838,0.016011


In [25]:
pred = aml.predict(test)
pred.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
0,0.727874,0.272126
0,0.707479,0.292521
0,0.636903,0.363097
1,0.304194,0.695806
0,0.676923,0.323077
1,0.266163,0.733837
1,0.27689,0.72311
1,0.620423,0.379577
0,0.671162,0.328838
0,0.826923,0.173077


In [0]:
test